<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/18_dataset_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset & DataLoader

In [ ]:
import torch
import numpy as np

from torch.utils.data import Dataset, DataLoader
from collections.abc import Iterator, Iterable, Sized

## Dataset
- PyTorch 의 tensor 와 학습에 사용될 일반 raw data (흔히, storage에 저장된 파일들) 사이에 위치.
- raw-data로부터 PyTorch의 기본데이터형 인 tensor를 얻게 해주는 역할을 수행하는 class.

## Custom Dataset 만들기
torch.uitl.data 모듈의 Dataset을 상속하고,
다음의 methods를 overriding해야 함.

###`__init__(self)`
- Dataset 인스턴스에 대한 **생성자**로 데이터셋에 대한 초기화를 담당.
- raw-data에 따라 **parameters를 자유롭게 추가**할 수 있음.

###`__len__(self)`
- Dataset 인스턴스 내에 있는 샘플 갯수를 반환하도록 구현.

###`__getitem__(self, idx)`
- argument로 넘어오는 idx 에 해당하는 샘플을 반환하도록 구현.
- `dataset[idx]` 등으로 Dataset 인스턴스인 dataset에 대해 idx에 해당하는 데이터셋 샘플에 접근할 때 호출됨.
- 하나의 샘플은 input feature data와 label을 묶어서 tuple로 반환.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data).float()
        self.labels = torch.tensor(labels).float()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

x = [ 0.5, 14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] # data
y = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # labels

dataset = CustomDataset(x, y)

In [ ]:
CustomDataset.mro()

[__main__.CustomDataset,
 torch.utils.data.dataset.Dataset,
 typing.Generic,
 object]

In [ ]:
print(isinstance(dataset, Iterator))
print(isinstance(dataset, Dataset))

False
True


In [ ]:
i = iter(dataset)
print(next(i))
print(i.__next__())

(tensor(0.5000), tensor(35.7000))
(tensor(14.), tensor(55.9000))


In [ ]:
# absolute base class
class sub_sized(Sized):
  def __len__(self):
    print('=====')
    return 10

In [ ]:
a = sub_sized()
len(a)

=====


10

## DataLoader

Dataset을 통해 데이터 로딩을 수행하는 Class.

Dataset으로부터 실제 Training Loop등에 training (mini)batch를 묶어서 효율적으로 제공해주는 역할을 수행

- data loading: raw-data로부터 tensor 를 얻어내는 과정 수행.
- 병렬 처리: storage에서 데이터를 읽어들이는 loading
은 속도가 느리므로 병렬처리
- shuffle: training 에서 데이터 샘플의 순서를 섞음.


DataLoader는 Dataset 을 제공해 줄 경우,
해당 Dataset을 통해 이루어지는 데이터 로딩을
병렬처리 및 shuffle, minibatch로 나누는 기능 등을 추가하여 수행하도록 도와줌.

```
data_loader = DataLoader(
    dataset,     # torch.utils.data.Dataset의 instance
    batch_size,  # batch의 샘플수
    shffule,     # boolean, 셔플링을 할지 여부(순서를 랜덤하게)
    num_workers, # 데이터로딩에 사용되는 sub-process의 수 (CPU의 core수를 넘으면 안됨.)
    pin_memory,  # boolean, GPU memory 영역을 예약할지 여부(pin).
    drop_last,   # boolean, 마지막 batch가 샘플의 수가 맞지 않을 경우 drop할지 여부.
    )
```

In [ ]:
data_loader = DataLoader(
    dataset,
    batch_size = 4,
    shuffle = True,
)

for batch_idx, (data, labels) in enumerate(data_loader):
    print(f'{batch_idx=}')
    print(f'{data.shape} | {data=}')
    print(f'{labels.shape} | {labels=}')

    # training ...

batch_idx=0
torch.Size([4]) | data=tensor([ 0.5000, 21.0000, 15.0000, 13.0000])
torch.Size([4]) | labels=tensor([35.7000, 68.4000, 58.2000, 60.4000])
batch_idx=1
torch.Size([4]) | data=tensor([-4., 11., 14.,  3.])
torch.Size([4]) | labels=tensor([21.8000, 56.3000, 55.9000, 33.9000])
batch_idx=2
torch.Size([3]) | data=tensor([28.,  8.,  6.])
torch.Size([3]) | labels=tensor([81.9000, 48.9000, 48.4000])
